In [2]:
import pandas as pd
import mysql.connector

from IPython.display import display, Markdown, HTML

db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="secret",
    port="3306",
    database="serlo"
)

events = pd.read_sql("""
    select
        edit.date as date_of_edit,
        review.date as date_of_review,
        user.username as reviewer
    from event_log edit
        join event_log review on review.uuid_id = edit.uuid_id
        join user on user.id = review.actor_id
    where
        edit.event_id = 5
        and review.event_id in (6,11)
        and edit.actor_id != review.actor_id;
""", db)
events.head()

/home/kulla/.local/share/asdf/installs/python/3.9.1/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


,date_of_edit,date_of_review,reviewer
0,2014-03-03 13:33:14,2014-03-03 13:37:34,Simon
1,2014-03-04 11:00:41,2014-03-04 12:10:52,arekkas
2,2014-03-04 11:12:51,2014-03-04 12:21:28,arekkas
3,2014-03-04 11:00:41,2014-03-04 12:57:43,wasmeier
4,2014-03-03 09:40:11,2014-03-04 12:57:56,Hannes


# Reviewerinnen mit maximal Anzahl an Reviews in einem 90 Tagezeitraum in den letzten 365 Tagen

In [29]:
df = events[events["date_of_review"] > pd.datetime.now() - pd.Timedelta("1Y")]
df = df[["date_of_review", "reviewer"]]

def agg_func(data):
    d = data.set_index("date_of_review")
    d = d.resample("D").count()
    d.fillna(0, inplace=True)
    return int(d.rolling(90, min_periods=1).sum().max())

df = df.groupby("reviewer").agg(agg_func)
df.sort_values("date_of_review", ascending=False, inplace=True)
df.rename(columns={"date_of_review": "max reviews in 90day periods"}, inplace=True)

display(df)
display("Count: %s" % len(df))

<ipython-input-29-aa2f2155d86e>:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  df = events[events["date_of_review"] > pd.datetime.now() - pd.Timedelta("1Y")]
/home/kulla/.local/share/asdf/installs/python/3.9.1/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3427: FutureWarning: Units 'M', 'Y' and 'y' do not represent unambiguous timedelta values and will be removed in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)


,max reviews in 90day periods
reviewer,
Sascha_Lill_95,793
Peter,637
Karin,546
WandaPaetzold,524
metzgaria,293
kathongi,221
Nish,146
wolfgang,85
Nanami,82


'Count: 27'

# Anzahl aktive Reviewer:innen (10 Reviews in einem 90 Tage-Zeitraum) 

In [30]:
display(df[df["max reviews in 90day periods"] > 10])
display("Count: %s" % len(df[df["max reviews in 90day periods"] > 10]))

,max reviews in 90day periods
reviewer,
Sascha_Lill_95,793
Peter,637
Karin,546
WandaPaetzold,524
metzgaria,293
kathongi,221
Nish,146
wolfgang,85
Nanami,82


'Count: 16'

# Anzahl sehr aktive Reviewer:innen (100Reviews in einem 90 Tage-Zeitraum) 

In [31]:
display(df[df["max reviews in 90day periods"] > 100])
display("Count: %s" % len(df[df["max reviews in 90day periods"] > 100]))

,max reviews in 90day periods
reviewer,
Sascha_Lill_95,793
Peter,637
Karin,546
WandaPaetzold,524
metzgaria,293
kathongi,221
Nish,146


'Count: 7'

# Reviewer:innen im letzten Jahr

In [18]:
df = events[events["date_of_review"] > pd.datetime.now() - pd.Timedelta("1Y")]
df = df.groupby("reviewer").count()
df.rename(columns={"date_of_review":"review_count"}, inplace=True)
df.sort_values("review_count", inplace=True, ascending=False)

display(df)
display("Count: %s" % len(df))

<ipython-input-18-166a39084db7>:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  df = events[events["date_of_review"] > pd.datetime.now() - pd.Timedelta("1Y")]
/home/kulla/.local/share/asdf/installs/python/3.9.1/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3427: FutureWarning: Units 'M', 'Y' and 'y' do not represent unambiguous timedelta values and will be removed in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)


,date_of_edit,review_count
reviewer,,
Karin,1717,1717
Peter,1443,1443
Sascha_Lill_95,796,796
WandaPaetzold,530,530
metzgaria,413,413
kathongi,345,345
Nish,190,190
Nanami,181,181
Gesine,101,101


'Count: 27'

# Aktive Reviewer:innen im letzten Jahr

In [17]:
display(df[df["review_count"] > 10])
display("count: %s" % len(df[df["review_count"] > 10]))

,date_of_edit,review_count
reviewer,,
Karin,1717,1717
Peter,1443,1443
Sascha_Lill_95,796,796
WandaPaetzold,530,530
metzgaria,413,413
kathongi,345,345
Nish,190,190
Nanami,181,181
Gesine,101,101


'count: 16'

# Sehr aktive Reviewer:innen (> 100 Reviews in den letzten 365 Tagen)

In [19]:
display(df[df["review_count"] > 100])
display("count: %s" % len(df[df["review_count"] > 100]))

,date_of_edit,review_count
reviewer,,
Karin,1717,1717
Peter,1443,1443
Sascha_Lill_95,796,796
WandaPaetzold,530,530
metzgaria,413,413
kathongi,345,345
Nish,190,190
Nanami,181,181
Gesine,101,101


'count: 9'

# Reviewzeiten in den letzten 365 Tagen (letztes Jahr)

In [21]:
d= events[events["date_of_review"] > pd.datetime.now() - pd.Timedelta("1Y")]
d= d["date_of_review"] - d["date_of_edit"]
d.describe()

<ipython-input-21-97e5e89bb2a4>:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  d= events[events["date_of_review"] > pd.datetime.now() - pd.Timedelta("1Y")]


count                          6108
mean      5 days 15:53:27.872626064
std      40 days 02:35:02.058048945
min                 0 days 00:00:33
25%          0 days 22:35:56.250000
50%          2 days 08:37:04.500000
75%          5 days 02:35:56.750000
max              2879 days 12:09:45
dtype: object

## 50/75/95%-Perzentile

In [23]:
d.quantile([0.5,0.75,0.95])

0.50       2 days 08:37:04.500000
0.75       5 days 02:35:56.750000
0.95   16 days 03:13:49.149999999
dtype: timedelta64[ns]

# Anzahl neuer Reviewer:innen (Personen, die in den letzten 365 Tagen ihre erstes Review gemacht haben)

In [27]:
a = events[events["date_of_review"] > pd.datetime.now() - pd.Timedelta("1Y")]
b = events[events["date_of_review"] <= pd.datetime.now() - pd.Timedelta("1Y")]

set(a["reviewer"]) - set(b["reviewer"])

<ipython-input-27-aa10dc26cdf0>:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  a = events[events["date_of_review"] > pd.datetime.now() - pd.Timedelta("1Y")]
<ipython-input-27-aa10dc26cdf0>:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  b = events[events["date_of_review"] <= pd.datetime.now() - pd.Timedelta("1Y")]


{'Annika-Hemlein',
 'Corinna',
 'Felix_Eccardt',
 'Kowalsky',
 'Sascha_Lill_95',
 'opamp',
 'sivakumar'}